In [3]:
#Importing packages I think I might need

import pandas as pd
from sql import *
import datetime
from time import strftime
from datetime import datetime
import numpy as np

In [138]:
pwd

u'/Users/megara'

In [5]:
#reading in file 1
engagement=pd.read_csv('Documents/take_home/takehome_user_engagement.csv')

In [6]:
#reading in file 2
users=pd.read_csv('Documents/take_home/takehome_users.csv')

In [139]:
#inspecting data
engagement.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [8]:
#inspecting data
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1398138810,1,0,11,10803
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1396237504,0,0,1,316
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1363734892,0,0,94,1525
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1369210168,0,0,1,5151
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1358849660,0,0,193,5240


In [9]:
#Joining on tables with an outer join to retain any users that have no engagement. 
#(turns out thats not totally necessary because everyone logged at least once)
user_engagement=users.merge(engagement, how='outer', left_on='object_id', right_on='user_id')

In [10]:
user_engagement.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,time_stamp,user_id,visited
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1398138810,1,0,11,10803,2014-04-22 03:53:30,1,1
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1396237504,0,0,1,316,2013-11-15 03:45:04,2,1
2,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1396237504,0,0,1,316,2013-11-29 03:45:04,2,1
3,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1396237504,0,0,1,316,2013-12-09 03:45:04,2,1
4,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1396237504,0,0,1,316,2013-12-25 03:45:04,2,1


In [11]:
user_engagement.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 211094 entries, 0 to 211093
Data columns (total 13 columns):
object_id                     211094 non-null int64
creation_time                 211094 non-null object
name                          211094 non-null object
email                         211094 non-null object
creation_source               211094 non-null object
last_session_creation_time    207917 non-null float64
opted_in_to_mailing_list      211094 non-null int64
enabled_for_marketing_drip    211094 non-null int64
org_id                        211094 non-null int64
invited_by_user_id            118528 non-null float64
time_stamp                    207917 non-null object
user_id                       207917 non-null float64
visited                       207917 non-null float64
dtypes: float64(4), int64(4), object(5)
memory usage: 22.5+ MB


In [12]:
users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 0 to 11999
Data columns (total 10 columns):
object_id                     12000 non-null int64
creation_time                 12000 non-null object
name                          12000 non-null object
email                         12000 non-null object
creation_source               12000 non-null object
last_session_creation_time    8823 non-null float64
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            6417 non-null float64
dtypes: float64(2), int64(4), object(4)
memory usage: 1.0+ MB


In [13]:
engagement.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 207917 entries, 0 to 207916
Data columns (total 3 columns):
time_stamp    207917 non-null object
user_id       207917 non-null int64
visited       207917 non-null int64
dtypes: int64(2), object(1)
memory usage: 6.3+ MB


In [19]:
#Grouping by number of logins, this way I can filter out the users who haven't even logged in 3 times. 
user_visits_count=user_engagement.groupby('object_id').count()

In [20]:
#filtering on users that haven't logged in at least three times
user_min_count_3=user_visits_count[user_visits_count.visited >= 3]

In [21]:
user_min_count_3.reset_index(inplace=True)

In [22]:
engaged_users=user_min_count_3.object_id

In [23]:
engaged_users_engagement=user_engagement.merge(pd.DataFrame(engaged_users), how='inner')

In [24]:
#writing a function that will return the week number and year from the login timestamp
def apply_week(x): return datetime.fromtimestamp(x).strftime('%W %y')


In [26]:
#just checking to make sure it works!
apply_week(engaged_users_engagement['last_session_creation_time'][1])

'12 14'

In [28]:
#lets drop everyone who hasn't logged in
engaged_users_engagement=engaged_users_engagement[pd.notnull(engaged_users_engagement['last_session_creation_time'])]

In [30]:
engaged_users_engagement.reset_index(inplace=True)

In [31]:
#creating a new column for the week and year of each login so I can group by on this field later
engaged_users_engagement['week']=np.nan
for i in range(len(engaged_users_engagement)):
    engaged_users_engagement.loc[i,'week']=apply_week(engaged_users_engagement.last_session_creation_time[i])


In [47]:
# grouping by user and week to get a count of how many times they logged in in each week
count_week=engaged_users_engagement.groupby(['week','object_id']).count()

In [51]:
count_week.reset_index(inplace=True)

In [53]:
#filtering on a minimum of 3 logins in a week
count_week_3=count_week[count_week.index>=3]

In [54]:
#checking to see if there was a difference between the users who'd logged in at least 3 times and logged in at least 3
#times in one week. not a huge diffence! 
len(count_week_3), len(count_week)

(2245, 2248)

In [58]:
#getting a list of all the distinct active users
active_users=count_week_3.object_id.unique()

In [62]:
#adding a column to users to indicate whether they are active or not
users['active']=np.nan
for i in range(len(users)):
    if users.object_id[i] in active_users:
        users.loc[i,'active']=1
    else:
        users.loc[i,'active']=0

In [97]:
#model doesn't like nan so inserting another error value
users=users.fillna(-999)

In [98]:
#creating a column for whether or not a user was invited based on the error value I inserted above
users['invited']=np.nan
for i in range(len(users)):
    if users.invited_by_user_id[i] !=-999:
        users.loc[i,'invited']=1
    else:
        users.loc[i,'invited']=0

In [166]:
#grouping by org_id to get the total number of organzation members
org=users.groupby(['org_id']).count()

In [149]:
#creating a dataframe of referrers ids and their organizations
ref_group=users[['object_id','org_id']]

In [150]:
#making unique names for this new dfs columns
ref_group.columns = ['ref_id', 'ref_org_id']

In [208]:
#merging on the referrers id 
org_users=users.merge(ref_group, how='left', left_on='invited_by_user_id', right_on='ref_id')

In [209]:
#did I mess up the merge?? nope
len(org_users), len(users)

(12000, 12000)

In [210]:
org.reset_index(inplace=True)

In [212]:
#lets grab a df of the active users
active_users=users[users.active==1]

In [213]:
#and then do a group by to get the count of the active users in each org
org_active=active_users.groupby(['org_id']).count()

In [214]:
#unique column names for this new df
org_active.reset_index(inplace=True)
org_active=org_active[['org_id','name']]
org_active.columns=[['org_id','active_member_count']]

In [215]:
#merging on org_id so I get a new column with the number of org members in the org of each user
org_users_1=org_users.merge(org, how='left', on='org_id')

In [216]:
#merging on org_id so I get a new column with the number of active org members in the org of each user
org_users_2=org_users_1.merge(org_active, how='left', on='org_id')

In [218]:
#still the right number of users??
len(org_users_2)

12000

In [219]:
#lets check out my columns so I can see which would be useful in the classification model and which would cause "leakage"
org_users_2.columns

Index([u'object_id', u'creation_time', u'name', u'email', u'creation_source', u'last_session_creation_time', u'opted_in_to_mailing_list', u'enabled_for_marketing_drip', u'org_id', u'invited_by_user_id', u'active', u'invited', u'month', u'ref_id', u'ref_org_id', u'index', u'member_count', u'active_member_count'], dtype='object')

In [222]:
#new column for whether or not the invitee is from the same org as the invited
org_users_2['org_invite']=np.nan
for i in range(len(org_users)):
    if org_users_2.org_id[i]==org_users_2.ref_org_id[i]:
        org_users_2.loc[i,'org_invite']=1
    else:
        org_users_2.loc[i,'org_invite']=0

In [242]:
#dataframe of users and whether they are active
using_users=users[['object_id','active']]

In [247]:
#unique column names
using_users.columns=['ref_id','ref_active']

In [256]:
#merging on invitee id to get a new field showing whether or not the users inviter is active
org_users_3=org_users_2.merge(using_users, how='left', left_on='invited_by_user_id', right_on='ref_id')

In [257]:
#filling 0 for those who werent invited
org_users_3.fillna(0,inplace=True)

In [245]:
#filling 0 for those who weren't invited
org_users_2.fillna(0,inplace=True)

In [328]:
#grabbing features for the x data and exlcuding those that would cause leakage and generating dummies for categorical features
x_data=org_users_3[['creation_source','opted_in_to_mailing_list','enabled_for_marketing_drip','invited', 'org_invite','member_count','active_member_count','ref_active']]
X_data = pd.get_dummies(x_data)

In [329]:
y_data=org_users_3['active']

In [336]:
#training my model, using a larger sample for training to try to ensure I get enough active users to train on
#otherwise model just preducts not active
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, random_state=12, test_size=0.45)
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.34425925925925926

In [337]:
#confusion matrix showing the false pos, true pos, false negs, and true negs. Not a great model but still could
# provide some insight
from sklearn.metrics import confusion_matrix, classification_report
y_pred = clf.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[1067, 3330],
       [ 211,  792]])

In [340]:
#Coef returns the weights assigned to the features. The most important features being the ones farthest from zero
#the most postive indicated user more likely to be active and the most negative the user more likely to be non-active
pd.DataFrame(zip(X_data.columns, np.transpose(clf.coef_))).sort(1, ascending=False)

,0,1
5,active_member_count,[0.068776179356]
7,creation_source_GUEST_INVITE,[0.0498705339701]
6,ref_active,[0.0309906785367]
4,member_count,[0.0166941245654]
1,enabled_for_marketing_drip,[-0.0047292441151]
0,opted_in_to_mailing_list,[-0.00628239956858]
2,invited,[-0.0260215416809]
3,org_invite,[-0.0260215416809]
11,creation_source_SIGNUP_GOOGLE_AUTH,[-0.0595048609096]
8,creation_source_ORG_INVITE,[-0.075892075651]
